In [1]:
import pandas as pd

df = pd.read_parquet("/kaggle/input/wikinews-data-converter-final-stage-2/enwikinews-pages-revdocs.parquet")
df

,revision_id,page_id,page_namespace,page_title,page_text,last_update_timestamp,page_dates,page_text_extract_result
0,4516743,736,0,President of China lunches with Brazilian Pres...,"{{date|November 13, 2004}}\n{{Brazil}}\n\n{{w|...",2019-09-28T09:51:53Z,"[{{date|november 13, 2004}}]","{""batchcomplete"":"""",""query"":{""pages"":{""736"":{""..."
1,4516759,741,0,Palestinians to elect new president on January 9,[[File:Mahmoud abbas.jpg|frame|left|Mahmoud Ab...,2019-09-28T10:45:51Z,"[{{byline|date=november 14, 2004|location=[[w:...","{""batchcomplete"":"""",""query"":{""pages"":{""741"":{""..."
2,2280888,743,0,Brazilian delegation returns from Arafat funeral,"{{date|November 13, 2004}}\n{{Palestine}}The d...",2014-01-02T19:36:05Z,"[{{date|november 13, 2004}}]","{""batchcomplete"":"""",""query"":{""pages"":{""743"":{""..."
3,4516758,764,0,Hearing begins over David Hookes death,"{{Crime and law}}{{byline|date=November 15, 20...",2019-09-28T10:39:36Z,"[{{byline|date=november 15, 2004|location=[[me...","{""batchcomplete"":"""",""query"":{""pages"":{""764"":{""..."
4,1973838,779,0,Iran close to decision on nuclear program,"{{date|November 13, 2004}}\n{{Iran nuclear pro...",2013-08-21T16:07:41Z,"[{{date|november 13, 2004}}]","{""batchcomplete"":"""",""query"":{""pages"":{""779"":{""..."
...,...,...,...,...,...,...,...,...
21652,4804529,3003708,0,"Man drives car into crowd of people, killing 3...","{{develop}}\n{{date|November 12, 2024}} <!--le...",2024-11-12T17:32:29Z,"[{{date|november 12, 2024}}]","{""batchcomplete"":"""",""query"":{""badrevids"":{""480..."
21653,4805088,3003768,0,"Prison riot in Ecuador, at least 17 killed",{{tasks|src|npov|mos|re-review}}\n{{date|Novem...,2024-11-17T16:35:30Z,"[{{date|november 13, 2024}}]","{""batchcomplete"":"""",""query"":{""pages"":{""3003768..."
21654,4805240,3003827,0,2024 ARPS Conference,"{{develop}}\n{{date|November 12, 2024}}\n\nThe...",2024-11-19T12:19:00Z,"[{{date|november 12, 2024}}]","{""batchcomplete"":"""",""query"":{""pages"":{""3003827..."
21655,4805272,3003842,0,Japan's oldest Princess Yuriko the Princess Mi...,"{{tasks|src|re-review}}{{date|November 15, 202...",2024-11-19T18:23:32Z,"[{{date|november 15, 2024}}]","{""batchcomplete"":"""",""query"":{""pages"":{""3003842..."


In [2]:
import json
from tqdm.auto import tqdm

def parse_page_text(t):
    obj = json.loads(t["page_text_extract_result"])

    query = obj["query"]
    if "pages" not in query:
        print(query)
        return None
    else:
        query = query["pages"]
    query = query[next(iter(query.keys()))]

    assert query["pageid"] == t["page_id"]
    return query["extract"]
    

df["page_text_extract_result"] = [parse_page_text(x) for x in tqdm(list(df.to_dict("records")))]

  0%|          | 0/21657 [00:00<?, ?it/s]

{'badrevids': {'4805099': {'revid': 4805099, 'missing': ''}}}
{'badrevids': {'4804318': {'revid': 4804318, 'missing': ''}}}
{'badrevids': {'4803373': {'revid': 4803373, 'missing': ''}}}
{'badrevids': {'4802553': {'revid': 4802553, 'missing': ''}}}
{'badrevids': {'4802554': {'revid': 4802554, 'missing': ''}}}
{'badrevids': {'4802561': {'revid': 4802561, 'missing': ''}}}
{'badrevids': {'4802563': {'revid': 4802563, 'missing': ''}}}
{'badrevids': {'4803268': {'revid': 4803268, 'missing': ''}}}
{'badrevids': {'4803599': {'revid': 4803599, 'missing': ''}}}
{'badrevids': {'4804529': {'revid': 4804529, 'missing': ''}}}


In [3]:
import dateutil.parser as dparser
from datetime import datetime

date_format_priority = [
    "byline",
    "dateline",
    "date",
    "category"
]

def get_date_format_priority(date_string):
    for i, string in enumerate(date_format_priority):
        if string in date_string.lower():
            return i
    print(date_string)
    assert False

def parse_date(page_dates):
    page_dates_parsed = []
    for date_str in page_dates:
        if not dparser.parse(date_str, fuzzy=True) \
            == dparser.parse(date_str, yearfirst=True, fuzzy=True) \
            == dparser.parse(date_str, dayfirst=True, fuzzy=True):
            page_dates_parsed.append("ambiguous")
        else:
            page_dates_parsed.append(dparser.parse(date_str,fuzzy=True))
    
    if len(page_dates_parsed) > 1:
        if len(set(page_dates_parsed)) > 1:
            page_dates_format_priority = [get_date_format_priority(d) for d in page_dates]
            min_page_dates_format_priority = min(page_dates_format_priority)
            min_page_dates_format_priority_vals = [i for i in range(len(page_dates)) if page_dates_format_priority[i] == min_page_dates_format_priority]
            page_dates_parsed = [page_dates_parsed[i] for i in min_page_dates_format_priority_vals]
        else:
            page_dates_parsed = list(set(page_dates_parsed))

    page_dates_parsed = [(x.isoformat() if isinstance(x, datetime) else x) for x in page_dates_parsed]
    
    return page_dates_parsed

df["page_dates_parsed"] = [parse_date(x) for x in tqdm(df["page_dates"])]

  0%|          | 0/21657 [00:00<?, ?it/s]

In [4]:
print(f"# pages with zero dates: {len([x for x in df['page_dates_parsed'] if (len(x) == 0)])}")
print(f"# pages with ambiguous dates: {len([x for x in df['page_dates_parsed'] if ('ambiguous' in x)])}")
print(f"# pages with more than one date: {len([x for x in df['page_dates_parsed'] if (len(x) > 1)])}")

# pages with zero dates: 0
# pages with ambiguous dates: 1
# pages with more than one date: 16


In [5]:
df

,revision_id,page_id,page_namespace,page_title,page_text,last_update_timestamp,page_dates,page_text_extract_result,page_dates_parsed
0,4516743,736,0,President of China lunches with Brazilian Pres...,"{{date|November 13, 2004}}\n{{Brazil}}\n\n{{w|...",2019-09-28T09:51:53Z,"[{{date|november 13, 2004}}]","Saturday, November 13, 2004 \n\nHu Jintao, the...",[2024-11-13T00:00:00]
1,4516759,741,0,Palestinians to elect new president on January 9,[[File:Mahmoud abbas.jpg|frame|left|Mahmoud Ab...,2019-09-28T10:45:51Z,"[{{byline|date=november 14, 2004|location=[[w:...","Sunday, November 14, 2004 \nRAMALLAH — Acting ...",[2024-11-14T00:00:00]
2,2280888,743,0,Brazilian delegation returns from Arafat funeral,"{{date|November 13, 2004}}\n{{Palestine}}The d...",2014-01-02T19:36:05Z,"[{{date|november 13, 2004}}]","Saturday, November 13, 2004 \n\nThe delegation...",[2024-11-13T00:00:00]
3,4516758,764,0,Hearing begins over David Hookes death,"{{Crime and law}}{{byline|date=November 15, 20...",2019-09-28T10:39:36Z,"[{{byline|date=november 15, 2004|location=[[me...","Monday, November 15, 2004 \nMELBOURNE, Victori...",[2024-11-15T00:00:00]
4,1973838,779,0,Iran close to decision on nuclear program,"{{date|November 13, 2004}}\n{{Iran nuclear pro...",2013-08-21T16:07:41Z,"[{{date|november 13, 2004}}]","Saturday, November 13, 2004 \n\nIranian repres...",[2024-11-13T00:00:00]
...,...,...,...,...,...,...,...,...,...
21652,4804529,3003708,0,"Man drives car into crowd of people, killing 3...","{{develop}}\n{{date|November 12, 2024}} <!--le...",2024-11-12T17:32:29Z,"[{{date|november 12, 2024}}]",None,[2024-11-12T00:00:00]
21653,4805088,3003768,0,"Prison riot in Ecuador, at least 17 killed",{{tasks|src|npov|mos|re-review}}\n{{date|Novem...,2024-11-17T16:35:30Z,"[{{date|november 13, 2024}}]","Wednesday, November 13, 2024 \n\nAt least sev...",[2024-11-13T00:00:00]
21654,4805240,3003827,0,2024 ARPS Conference,"{{develop}}\n{{date|November 12, 2024}}\n\nThe...",2024-11-19T12:19:00Z,"[{{date|november 12, 2024}}]","Tuesday, November 12, 2024 \nThe Australasian ...",[2024-11-12T00:00:00]
21655,4805272,3003842,0,Japan's oldest Princess Yuriko the Princess Mi...,"{{tasks|src|re-review}}{{date|November 15, 202...",2024-11-19T18:23:32Z,"[{{date|november 15, 2024}}]","Wednesday, November 20, 2024 \n\nJapan's Yurik...",[2024-11-15T00:00:00]


In [6]:
df.to_parquet(
    "enwikinews-processed.parquet",
    engine='pyarrow', 
    compression='zstd', 
    compression_level=22)